In [3]:
# %% [markdown]
# # Fast Embedding Generation
# ![Optimized Pipeline](visuals/optimized_flow.png)

# %%
# Install only what's needed
!pip install -U --quiet langchain-huggingface sentence-transformers faiss-cpu

# %%
import os
import time
from pathlib import Path
import numpy as np
import pickle
from sentence_transformers import SentenceTransformer  # Direct usage for better performance
from langchain_community.vectorstores import FAISS

# Disable unnecessary warnings
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
os.environ['HF_HUB_DISABLE_SYMLINKS_WARNING'] = '1'

# %%
def fast_embed(chunks, model_name="all-MiniLM-L6-v2", batch_size=32):
    """Optimized embedding generation"""
    model = SentenceTransformer(model_name)
    texts = [c.page_content for c in chunks]
    return model.encode(texts, batch_size=batch_size, show_progress_bar=True)

# %%
# Main Execution
start_time = time.time()

try:
    # 1. Load processed chunks
    with open("../processed_data/chunks.pkl", "rb") as f:
        chunks = pickle.load(f)
    print(f"📂 Loaded {len(chunks)} chunks in {time.time()-start_time:.1f}s")
    
    # 2. Fast embedding
    embed_start = time.time()
    embeddings = fast_embed(chunks)
    print(f"⚡ Generated embeddings in {time.time()-embed_start:.1f}s")
    
    # 3. Create FAISS index
    index_start = time.time()
    db = FAISS.from_embeddings(
        text_embeddings=list(zip([c.page_content for c in chunks], embeddings)),
        embedding=SentenceTransformer("all-MiniLM-L6-v2")  # Wrapper for FAISS compatibility
    )
    db.save_local("../faiss_store")
    print(f"💾 Saved index in {time.time()-index_start:.1f}s")
    
    print(f"\n✅ Total time: {time.time()-start_time:.1f} seconds")
    print(f"🚀 Speed: {len(chunks)/(time.time()-start_time):.1f} docs/sec")

except Exception as e:
    print(f"❌ Error: {str(e)}")

📂 Loaded 276 chunks in 0.0s


Batches:   0%|          | 0/9 [00:00<?, ?it/s]

⚡ Generated embeddings in 8.1s


`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


💾 Saved index in 3.3s

✅ Total time: 11.4 seconds
🚀 Speed: 24.2 docs/sec


In [6]:
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

# Use embedding wrapper (not raw model)
embedding_function = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={"device": "cpu"}
)

# Load FAISS with correct embedding wrapper
db = FAISS.load_local(
    "../faiss_store",
    embeddings=embedding_function,
    allow_dangerous_deserialization=True
)

# Query
query = "What does this insurance policy cover?"
results = db.similarity_search(query, k=3)

# Display results
print("\n🔍 Top 3 results for query:")
for i, doc in enumerate(results):
    print(f"\n--- Result {i+1} ---\n{doc.page_content[:500]}...")



🔍 Top 3 results for query:

--- Result 1 ---
statements and any information in addition thereto supplied to the Company by the Insured or on  
Insured‟s behalf. 
 
4. “Insured Premises” means the place(s) declared for insurance and named in the Schedule attached 
to the Policy. 
 
5. “Policy” means the Policy Booklet, the Schedule, the Proposal and any applicable endorsements or 
memoranda. The Policy contains the details of the extent of the cover available to the Insured, what 
is excluded from the cover and the conditions, warranties ba...

--- Result 2 ---
Loss, and We pay You the full Sum Insured for such item, the insurance cover  for that item will 
automatically end. If We pay the total Sum Insured for any claim, this Policy will end. 
f. Effect of death: If You are an individual, in the event of Your unfortunate death, the Insurance 
Covers that You have purchased will continue for the benefit of Your legal representative/s during 
the Policy Period subject to all the terms a